In [ ]:
## 1st-C6AI-PoC--ElectionGuard-Web-API-in-Python-using-FastAPI-231022b.ipynb
## Commentable @ https://colab.research.google.com/drive/1l3vriAwsCXHfA3nsQqXzoPnwBknFIm8i?usp=sharing
## Editable @ https://colab.research.google.com/drive/1l3vriAwsCXHfA3nsQqXzoPnwBknFIm8i

In [ ]:
!pip install -q electionguard

In [ ]:
!pip install -q "fastapi[all]"

# lida 0.0.10 requires kaleido, which is not installed.
# llmx 0.0.15a0 requires cohere, which is not installed.
# llmx 0.0.15a0 requires openai, which is not installed.
# llmx 0.0.15a0 requires tiktoken, which is not installed.


In [ ]:
# Import FastAPI
from fastapi import FastAPI

In [ ]:
# Import electionguard-python
from electionguard.ballot import CiphertextBallot
from electionguard.election import CiphertextElectionContext
from electionguard.encrypt import encrypt_ballot
from electionguard.group import ElementModQ
from electionguard.guardian import Guardian
from electionguard.key_ceremony import CeremonyDetails
from electionguard.manifest import InternalManifest
from electionguard.tally import CiphertextTally


In [ ]:
# Create a FastAPI app
app = FastAPI()


In [ ]:
# Define some global variables for the election
NUM_GUARDIANS = 3 # The number of guardians in the election
QUORUM = 2 # The quorum of guardians required to decrypt the tally
GUARDIANS = [] # A list of guardians
ELECTION_KEYS = None # The public keys of the guardians
ELECTION_MANIFEST = None # The manifest of the election
ELECTION_CONTEXT = None # The context of the encryption
BALLOTS = [] # A list of encrypted ballots
TALLY = None # The encrypted tally


In [ ]:
# Define some helper functions for the key ceremony
def create_guardians():
    """Create the guardians for the key ceremony"""
    global GUARDIANS
    GUARDIANS = [
        Guardian(f"guardian-{i}", i, NUM_GUARDIANS, QUORUM)
        for i in range(NUM_GUARDIANS)
    ]

def share_guardian_public_keys():
    """Share the public keys of the guardians"""
    global ELECTION_KEYS
    ELECTION_KEYS = {
        guardian.object_id: guardian.share_public_keys()
        for guardian in GUARDIANS
    }

def generate_election_partial_key_backups():
    """Generate the partial key backups for each guardian"""
    for guardian in GUARDIANS:
        guardian.generate_election_partial_key_backups(ELECTION_KEYS)

def share_election_partial_key_backups():
    """Share the partial key backups with other guardians"""
    for guardian in GUARDIANS:
        for other_guardian in GUARDIANS:
            if other_guardian.object_id != guardian.object_id:
                backup = guardian.share_election_partial_key_backup(other_guardian.object_id)
                other_guardian.save_election_partial_key_backup(backup)

def verify_election_partial_key_backups():
    """Verify the partial key backups"""
    for guardian in GUARDIANS:
        for other_guardian in GUARDIANS:
            if other_guardian.object_id != guardian.object_id:
                backup = other_guardian.get_election_partial_key_backup(guardian.object_id)
                verification = guardian.verify_election_partial_key_backup(backup)
                other_guardian.save_election_partial_key_verification(verification)

def publish_joint_public_key():
    """Publish the joint public key of the guardians"""
    global ELECTION_CONTEXT
    joint_public_key = Guardian.combine_election_public_keys(ELECTION_KEYS)
    ELECTION_CONTEXT = CiphertextElectionContext(
        number_of_guardians=NUM_GUARDIANS,
        quorum=QUORUM,
        elgamal_public_key=joint_public_key,
        commitment_hash=ELECTION_MANIFEST.crypto_hash(),
        manifest_hash=ELECTION_MANIFEST.crypto_hash(),
        crypto_base_hash=ELECTION_MANIFEST.crypto_base_hash,
        crypto_extended_base_hash=ELECTION_MANIFEST.crypto_extended_base_hash,
    )


In [ ]:
# Define some helper functions for the ballot encryption
def encrypt_ballot(ballot):
    """Encrypt a ballot using the election manifest and context"""
    encrypted_ballot = encrypt_ballot(
        ballot, ELECTION_MANIFEST, ELECTION_CONTEXT
    )
    return encrypted_ballot


In [ ]:
# Define some helper functions for the tallying
def create_tally():
    """Create a tally for the election"""
    global TALLY
    TALLY = CiphertextTally("tally", ELECTION_MANIFEST, ELECTION_CONTEXT)

def append_ballot_to_tally(ballot):
    """Append a ballot to the tally"""
    global TALLY
    TALLY.append(ballot)

def decrypt_tally():
    """Decrypt the tally using the guardians' shares"""
    global TALLY
    shares = {}
    for guardian in GUARDIANS:
        shares[guardian.object_id] = guardian.share_election_public_key()
        for other_guardian in GUARDIANS:
            if other_guardian.object_id != guardian.object_id:
                backup = other_guardian.get_election_partial_key_backup(guardian.object_id)
                share = guardian.compute_election_partial_key_challenge(backup, ELECTION_CONTEXT)
                shares[guardian.object_id].add_partial_key_share(share)
    decrypted_tally = TALLY.decrypt_with_shares(shares)
    return decrypted_tally


### microsoft.github.io/electionguard-python/#requirements

In [ ]:
#TODO: ?

## !make environment

## !make start API_MODE=mediator

### or
## !make start API_MODE=guardian


In [ ]:
# #TODO: ?
# ## https://hub.docker.com/settings/security

# # Pull the image from DockerHub
# docker pull docker.pkg.github.com/microsoft/electionguard-web-api/electionguard-web-api:main


In [ ]:
# # Start a container for the API in mediator mode, exposed on port 80 of the host machine
# docker run -d -p 80:8000 --env API_MODE=mediator docker.pkg.github.com/microsoft/electionguard-web-api/electionguard-web-api:main


In [ ]:
## c/o: https://microsoft.github.io/electionguard-api-python/#quick-start

## APIs
"""
application can run in one of two modes:

guardian mode runs features used by Guardians (key ceremony actions, partial tally decryption, etc.)
mediator mode runs features used by Mediators (ballot encryption, casting, spoiling, etc.)
In practice, you will likely need to run at least one instance of each mode. We provide a single codebase and Docker image, but the mode can be set at runtime.
 """

## Running with Python
"""
Quick Start
Using make, installation and startup can be run with one command:

To set up the environment:

make environment
To start the api:

make start API_MODE=mediator
OR

make start API_MODE=guardian
 """

## Documentation
"""
FastApi defaultly has API documentation built in. The following is available after running:

SwaggerUI at http://127.0.0.1:8000/docs or http://127.0.0.1:8001/docs, depending on the API mode

ReDoc at http://127.0.0.1:8000/redoc or http://127.0.0.1:8001/redoc
 """

## Debugging
"""
For local debugging with Visual Studio Code, choose the Guardian Web API or Mediator Web API options from the dropdown in the Run menu. Once the server is up, you can easily hit your breakpoints.

If the code fails to run, make sure your Python interpreter is set to use your poetry environment.
 """

## Testing
"""
A Postman collection is available to test the API located in the /tests folder. You can do a few things with this:

Import into Postman for easy manual testing.
Run locally with the Newman CLI.
Run the APIs and tests entirely in Docker by running: bash
  make docker-test
 """

## Docker
"""
Running with Docker
The official Docker image
We host a Docker image on both Github Packages and DockerHub.

Note: GitHub Packages requires authentication to retrieve the package. This requires a GitHub Access Token and using docker login. Follow GitHub instructions.

# Pull the image from DockerHub
docker pull docker.pkg.github.com/microsoft/electionguard-web-api/electionguard-web-api:main

# Start a container for the API in mediator mode, exposed on port 80 of the host machine
docker run -d -p 80:8000 --env API_MODE=mediator docker.pkg.github.com/microsoft/electionguard-web-api/electionguard-web-api:main
OR

# Pull the image from DockerHub
docker pull electionguard/electionguard-web-api:latest

# Start a container for the API in mediator mode, exposed on port 80 of the host machine
docker run -d -p 80:8000 --env API_MODE=mediator electionguard/electionguard-web-api:latest
Developing locally with Docker
If you run Docker and want to run the code locally without Python dependencies, we provide a Dockerfile and docker-compose.yml.

Run both APIs at the same time:

make docker-run
Or run both APIs in development mode, with automatic reloading on file change:

make docker-dev
After either command, you will find the mediator API running at http://127.0.0.1:8000 and the guardian API at http://127.0.0.1:8001
 """

print()


In [ ]:
# # Define some API endpoints for the election process
# @app.post("/create_election")
# def create_election(manifest: InternalManifest):
#     """Create an election with a given manifest"""
#     global ELECTION_MANIFEST
#     ELECTION_MANIFEST = manifest
#     # Perform the key ceremony
#     create_guardians()
#     share_guardian_public_keys()
#     generate_election_partial_key_backups()
#     share_election_partial_key_backups()
#     verify_election_partial_key_backups()
#     publish_joint_public_key()
#     # Return the election context
#     return ELECTION_CONTEXT


In [ ]:

# @app.post("/encrypt_ballot")
# def encrypt_ballot(ballot: CiphertextBallot):
#     """Encrypt a ballot with the election context"""
#     encrypted_ballot = encrypt_ballot(ballot)
#     return encrypted_ballot


In [ ]:

# @app.post("/cast_ballot")
# def cast_ballot(ballot: CiphertextBallot):
#     """Cast a ballot and append it to the tally"""
#     append_ballot_to_tally(ballot)
#     return {"status": "success"}


In [ ]:

# @app.post("/spoil_ballot")
# def spoil_ballot(ballot: CiphertextBallot):
#     """Spoil a ballot and do not append it to the tally"""
#     return {"status": "success"}


In [ ]:

# @app.get("/tally")
# def tally():
#     """Tally the results and decrypt them with the guardians' shares"""
#     decrypted_tally = decrypt_tally()
#     return decrypted_tally
